In [7]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')



In [8]:
from config_paper import key_id, secret_key
from cross_signal import *

In [9]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url )

In [10]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-04-27'

In [11]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [12]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'400000'

In [13]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [14]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.25

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [19]:
returneddata = api.get_barset('FLS',barTimeframe,limit=150).df
returneddata2=api.polygon.historic_agg_v2( 'XLNX', 1, 'minute',_from='2020-04-23', to=startdate,).df
#data_indic=populateindicators(returneddata['AMP'])


In [20]:
data=populateindicators(returneddata2)

In [21]:
cashBalance = account.cash

targetPositionSize = int(int(cashBalance) / (data['close'][-1] / positionSizing)) # Calculates required position size

                    

In [23]:
returned = api.submit_order('XLNX',targetPositionSize,"buy","market","day") # Market order to open position

In [53]:
 openPosition = api.get_position('CSCO')

APIError: position does not exist

In [49]:
int(openPosition.qty)

-562

In [51]:
api.submit_order('XLNX',int(562),"buy","market","day")

Order({   'asset_class': 'us_equity',
    'asset_id': '346931c8-f921-4834-87e6-a0505d5d0b5d',
    'canceled_at': None,
    'client_order_id': 'b9fc7f00-147c-4cc7-b4ef-cab6873e7496',
    'created_at': '2020-04-27T19:05:30.267229Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '7a86b890-d988-421a-ab2e-49342a5b18b2',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '562',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-27T19:05:30.249833Z',
    'symbol': 'XLNX',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-27T19:05:30.267229Z'})

In [ ]:
for minu in range(500) :
        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-20', to=startdate,).df
            #api.get_barset(symbol,barTimeframe,limit=150).df


            # Calculated trading indicators
            #populate exponential moving averages
            data=populateindicators(returned_data) 


             # Create the buy signal here
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['rsi'][-1] < 35 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] < data['plusdi'][-1]:


                cashBalance = account.cash

                targetPositionSize = int(int(cashBalance) / (data['close'][-1] / positionSizing)) # Calculates required position size

                returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                print(returned)


            elif data['adx'][-1] > 22 and \
            data['mfi'][-1] > 63 and \
            data['rsi'][-1] > 74 and \
            data['plusdi'][-1] > 25 and \
            data['minusdi'][-1] < data['plusdi'][-1]:
                # Closes position  SMA50
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                print(returned)
        
        except:
                print(symbol)
                
        #interval should be an integer, the number of seconds to wait
    time.sleep(20)

MMM
MO
BRKB
BFB
FTV
ITW
JCI
KSS
M
MKTK
MYL
NUE
ORCL
PWR
RTN
UTX

ANET
BRKB
BFB
CE
DD
FTV
JCI
MKTK
NUE
RTN
TSCO
UTX

MMM
APA
BRKB
BFB
DISCA
JCI
M
MKTK
